## Recreating the following for classification problem
https://www.tensorflow.org/text/tutorials/transformer

Need tensorflow-text for using the BERT model tokenizer

In [ ]:
# Install the nightly version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q -U tensorflow-text tensorflow

     |████████████████████████████████| 5.9 MB 14.5 MB/s 
     |████████████████████████████████| 578.0 MB 18 kB/s 
     |████████████████████████████████| 5.9 MB 55.5 MB/s 
     |████████████████████████████████| 1.7 MB 66.0 MB/s 
     |████████████████████████████████| 438 kB 79.5 MB/s 


Import the necessary modules:

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text
from sklearn.utils import shuffle 

Doing necessary data preprocessing

In [ ]:
os.listdir('.')

['.config', 'twitter_validation.csv', 'twitter_training.csv', 'sample_data']

In [ ]:
training_data_loc = "./twitter_training.csv"
val_data_loc = "./twitter_validation.csv"

In [ ]:
df = pd.read_csv(training_data_loc)
df

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [ ]:
df = df[df.columns[2:]]

In [ ]:
df = df.rename(columns={"Positive":"Sentiment", "im getting on borderlands and i will murder you all ,":"sentence"})

In [ ]:
df['Sentiment'].value_counts()

Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: Sentiment, dtype: int64

In [ ]:
df = df[df.apply(lambda x: (x["Sentiment"] == "Positive") or (x["Sentiment"] == "Negative"), axis=1)]

In [ ]:
df['Sentiment'].value_counts()

Negative    22542
Positive    20831
Name: Sentiment, dtype: int64

In [ ]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == "Positive" else 0)

In [ ]:
df = shuffle(df)

df

,Sentiment,sentence
64514,1,I love your list
6688,0,if you like this will regret it
46970,0,"Hey @JustinTrudeau , I read that the executive..."
53421,1,Maybe the greatest musical moment in a video w...
844,0,that
...,...,...
24701,1,Thank You @YouTubeIndia @YTCreatorsIndia for ...
11040,1,Remember that it was one of the best stealth g...
70094,0,@GhostRecon_UK fix your game. The Behemoth is ...
4307,0,my pc is crying


In [ ]:
df.isna().value_counts()

Sentiment  sentence
False      False       43012
           True          361
dtype: int64

In [ ]:
df.dropna(inplace=True) # dropping empty sentences

In [ ]:
df

,Sentiment,sentence
64514,1,I love your list
6688,0,if you like this will regret it
46970,0,"Hey @JustinTrudeau , I read that the executive..."
53421,1,Maybe the greatest musical moment in a video w...
844,0,that
...,...,...
24701,1,Thank You @YouTubeIndia @YTCreatorsIndia for ...
11040,1,Remember that it was one of the best stealth g...
70094,0,@GhostRecon_UK fix your game. The Behemoth is ...
4307,0,my pc is crying


In [ ]:
labels = df['Sentiment']
features = df['sentence']

## Tokenizer
using subword tokenizer implementation from Bert model

In [ ]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

184801/184801 [==============================] - 0s 0us/step


'./ted_hrlr_translate_pt_en_converter.zip'

In [ ]:
tokenizers = tf.saved_model.load(model_name)

In [ ]:
[item for item in dir(tokenizers) if not item.startswith('_')]

['en',
 'graph_debug_info',
 'pt',
 'signatures',
 'tensorflow_git_version',
 'tensorflow_version']

The `tf.saved_model` contains two text tokenizers, one for English and one for Portuguese. We just need the English one

In [ ]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [ ]:
sample = features.sample(n=5)
sample

49339    Shooting a lil weird, cos I was scoring from s...
61600    Just started getting back into GTA online beca...
71042    *More of me ogling <unk>. So immersive, so mus...
1595     @DuvalMagic @Borderlands game doesn't register...
67986    There is a 0% chance that this game will come ...
Name: sentence, dtype: object

In [ ]:
text_sample = sample.iloc[0]
print(text_sample)

Shooting a lil weird, cos I was scoring from some sick angles.. but i'm seeing something possible exciting with the passing


In [ ]:
encoded = tokenizers.en.tokenize(sample)
# print(encoded)
print('Padded and encoded embeddings:')
for row in encoded.to_list():
  print(row)

Padded and encoded embeddings:
[2, 3530, 37, 48, 1284, 2963, 13, 3455, 88, 45, 84, 55, 3284, 1241, 109, 147, 1907, 6047, 15, 15, 87, 45, 9, 49, 698, 173, 465, 1066, 93, 71, 5373, 3]
[2, 112, 232, 361, 193, 156, 43, 2297, 754, 120, 99, 709, 196, 4500, 6540, 89, 71, 718, 2776, 42, 376, 371, 15, 15, 15, 45, 9, 49, 131, 102, 181, 73, 528, 116, 56, 6558, 273, 72, 468, 75, 1831, 92, 3]
[2, 1, 121, 74, 114, 51, 651, 826, 1, 6363, 484, 1, 15, 82, 45, 4864, 2430, 13, 82, 1159, 14, 1090, 15, 45, 517, 57, 3283, 88, 376, 2200, 9, 55, 1612, 86, 2228, 13, 104, 140, 234, 15, 15, 15, 15, 81, 1477, 625, 28, 83, 727, 5022, 3564, 76, 15, 15, 15, 72, 115, 149, 37, 4252, 495, 93, 71, 2622, 4552, 2560, 77, 3]
[2, 31, 40, 795, 2566, 1361, 6440, 31, 2640, 3172, 1112, 196, 269, 9, 56, 4757, 169, 488, 13, 340, 4304, 105, 3730, 85, 121, 80, 269, 9, 56, 298, 341, 3672, 72, 488, 656, 38, 5567, 1723, 91, 269, 9, 56, 4757, 117, 45, 880, 6690, 27, 16, 1381, 1019, 52, 1607, 3079, 3079, 88, 3]
[2, 96, 80, 37, 17, 7, 10

The padded 2 and 3 stands for [START] and [END]

In [ ]:
encoded[:, :128]

<tf.RaggedTensor [[2, 3530, 37, 48, 1284, 2963, 13, 3455, 88, 45, 84, 55, 3284, 1241, 109,
  147, 1907, 6047, 15, 15, 87, 45, 9, 49, 698, 173, 465, 1066, 93, 71, 5373,
  3]                                                                        ,
 [2, 112, 232, 361, 193, 156, 43, 2297, 754, 120, 99, 709, 196, 4500, 6540,
  89, 71, 718, 2776, 42, 376, 371, 15, 15, 15, 45, 9, 49, 131, 102, 181, 73,
  528, 116, 56, 6558, 273, 72, 468, 75, 1831, 92, 3]                        ,
 [2, 1, 121, 74, 114, 51, 651, 826, 1, 6363, 484, 1, 15, 82, 45, 4864, 2430,
  13, 82, 1159, 14, 1090, 15, 45, 517, 57, 3283, 88, 376, 2200, 9, 55, 1612,
  86, 2228, 13, 104, 140, 234, 15, 15, 15, 15, 81, 1477, 625, 28, 83, 727,
  5022, 3564, 76, 15, 15, 15, 72, 115, 149, 37, 4252, 495, 93, 71, 2622,
  4552, 2560, 77, 3]                                                        ,
 [2, 31, 40, 795, 2566, 1361, 6440, 31, 2640, 3172, 1112, 196, 269, 9, 56,
  4757, 169, 488, 13, 340, 4304, 105, 3730, 85, 121, 80, 269, 9, 56,

In [ ]:
round_trip = tokenizers.en.detokenize(encoded[:, :128])

print('Original text:')
for line in round_trip.numpy():
  print(line.decode('utf-8'))

Original text:
shooting a lil weird , cos i was scoring from some sick angles . . but i ' m seeing something possible exciting with the passing
just started getting back into gta online because my god does rockstar have the incredible taste fo music . . . i ' m going at need to play these tunes again and keep that radio on
[UNK] more of me ogling [UNK] unk [UNK] . so immersive , so museum - quality . i hope ubisoft ' s artists are proud , people really should . . . . this urban environment ; they absolutely nailed it . . . and he did a spectacular job with the wilderness in
@ duvalmagic @ borderlands game doesn ' t register right sometimes , ammo regen for more isn ' t always working properly and sometimes entire bullets don ' t register when i hit enemies : / fix soon plssssss
there is a 0 % chance that this game will come out in 2020 . i can ' t wait to play it in march .


## Set up a data pipeline with `tf.data`

The following function takes batches of text as input, and converts them to a format suitable for training. 

1. It tokenizes them into ragged batches.
2. It trims each to be no longer than `MAX_TOKENS`.
3. It splits the output (English) tokens into inputs and labels. THese shifted by one step so that the `label` at each location is the id of the next token.
4. It converts the `RaggedTensor`s to padded dense `Tensor`s.
5. It returns an `(inputs, labels)` pair.

NOTE: We will bypass this method in the batch making area and utilize our own method since we are not using `tf.data`

# **Tokenizer**
Cannot prepare batches as required. I don't know how to fix it.

In [ ]:
MAX_TOKENS=128
def prepare_batch(en): # not actually preparing batches
    en = tokenizers.en.tokenize(en)
    en = en[:, :128] # pads with `.` if the sentence is less than 128 words and strips if more than 128 words
    # print(en)
    # en_inputs = en[:, :-1].to_tensor()
    return en

In [ ]:
tokenized_sentences = prepare_batch(features) # batch_size, 128

In [ ]:
list_tokenized = tokenized_sentences.to_list()

In [ ]:
len(list_tokenized), len(labels)

(43012, 43012)

In [ ]:
for i in range(len(list_tokenized)-43000):
  print(list_tokenized[i], labels.iloc[i])

[2, 45, 302, 135, 1261, 3] 1
[2, 107, 79, 106, 81, 153, 105, 651, 6526, 76, 3] 0
[2, 1366, 31, 112, 530, 271, 6531, 4829, 13, 45, 460, 75, 71, 6696, 5939, 1804, 555, 2689, 74, 5851, 4092, 1479, 8, 315, 2159, 601, 2007, 91, 98, 2486, 455, 49, 1107, 1360, 15, 81, 84, 3613, 159, 284, 37, 5231, 73, 1184, 15, 80, 81, 479, 30, 15, 119, 80, 81, 82, 30, 3310, 147, 5082, 89, 49, 1107, 1360, 92, 101, 2689, 13, 87, 147, 243, 2486, 71, 192, 30, 3] 0
[2, 285, 71, 1345, 3247, 368, 77, 37, 383, 59, 6525, 826, 1112, 478, 310, 15, 3] 1
[2, 75, 3] 0
[2, 38, 795, 6, 77, 230, 6, 6, 1065, 15, 192, 4134, 690, 394, 74, 135, 5252, 2502, 118, 40, 1058, 966, 5496, 1191, 128, 4944, 256, 48, 388, 3721, 685, 30, 3] 0
[2, 119, 71, 42, 2713, 145, 79, 98, 106, 765, 959, 5404, 19, 15, 45, 528, 81, 1112, 72, 45, 9, 49, 3866, 75, 76, 9, 55, 100, 205, 100, 76, 80, 15, 3] 1
[2, 31, 1470, 6401, 1163, 5140, 1572, 31, 1470, 6401, 1163, 35, 6957, 31, 57, 3283, 88, 376, 2200, 88, 1575, 5171, 31, 57, 3283, 88, 376, 2200, 31, 57

## Copied code 
we need to change it up a bit so we can use same definitions for our own classification problem

### Define the Embedding and Positional encoding

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
temp_encoding = positional_encoding(length=2048, depth=512)
temp_encoding

<tf.Tensor: shape=(2048, 512), dtype=float32, numpy=
array([[ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.84147096,  0.8218562 ,  0.8019618 , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.9092974 ,  0.9364147 ,  0.95814437, ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [ 0.17589758, -0.18608274, -0.7070546 , ...,  0.9741639 ,
         0.97595036,  0.97761387],
       [-0.7333133 ,  0.7014913 ,  0.1447375 , ...,  0.9741387 ,
         0.97592694,  0.97759205],
       [-0.9683193 ,  0.98535496,  0.8799798 , ...,  0.9741135 ,
         0.9759035 ,  0.9775702 ]], dtype=float32)>

In [ ]:
temp_encoding[tf.newaxis, :128, :].shape

TensorShape([1, 128, 512])

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x # (batch_size, length-of-signal, 512)


In [ ]:
x = np.zeros((400, 128, 512))
y = np.zeros((1, 128, 512))

In [ ]:
x = np.random.rand(4, 3, 2)
y = np.random.rand(1, 3, 2)


In [ ]:
x, y

(array([[[0.95609057, 0.60270507],
         [0.51131414, 0.71017839],
         [0.9538461 , 0.21263245]],
 
        [[0.6372833 , 0.14766658],
         [0.60528041, 0.24988174],
         [0.03645899, 0.9000626 ]],
 
        [[0.66726868, 0.68453458],
         [0.18968806, 0.86797481],
         [0.43211758, 0.7504659 ]],
 
        [[0.40709853, 0.90729841],
         [0.26381697, 0.56954289],
         [0.46140479, 0.01489403]]]), array([[[0.02513828, 0.58679921],
         [0.29969024, 0.72462405],
         [0.53957241, 0.83326605]]]))

In [ ]:
x+y

array([[[0.98122885, 1.18950428],
        [0.81100438, 1.43480244],
        [1.49341851, 1.0458985 ]],

       [[0.66242158, 0.73446579],
        [0.90497066, 0.97450579],
        [0.5760314 , 1.73332866]],

       [[0.69240696, 1.2713338 ],
        [0.4893783 , 1.59259886],
        [0.97168999, 1.58373195]],

       [[0.43223681, 1.49409762],
        [0.56350722, 1.29416694],
        [1.00097719, 0.84816008]]])

In [ ]:
tf.cast(512, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=512.0>

In [ ]:
tokenizers.en.get_vocab_size()

<tf.Tensor: shape=(), dtype=int32, numpy=7010>

In [ ]:
temp_layer = tf.keras.layers.Embedding(7010, 512, mask_zero=True)
# help(temp_layer)

In [ ]:
temp_model = tf.keras.Sequential()
temp_layer = tf.keras.layers.Embedding(1000, 64, input_length=10, mask_zero=True)
temp_model.add(temp_layer)
# The model will take as input an integer matrix of size (batch,
# input_length), and the largest integer (i.e. word index) in the input
# should be no larger than 999 (vocabulary size).
# Now model.output_shape is (None, 10, 64), where `None` is the batch
# dimension.
input_array = np.random.randint(1000, size=(1, 10))
temp_model.compile('rmsprop', 'mse')
output_array = temp_model.predict(input_array)
# print(output_array)
print(output_array.shape)
#(1, 10, 64)

1/1 [==============================] - 0s 62ms/step
(1, 10, 64)


In [ ]:
# print(temp_layer.compute_mask(input_array, mask_zero=True)) # would not work as this is just for forwarding the compute_mask

In [ ]:
embed_en = PositionalEmbedding(vocab_size=tokenizers.en.get_vocab_size(), d_model=512)
embed_en

In [ ]:
encoded.to_tensor()

<tf.Tensor: shape=(5, 68), dtype=int64, numpy=
array([[   2, 3530,   37,   48, 1284, 2963,   13, 3455,   88,   45,   84,
          55, 3284, 1241,  109,  147, 1907, 6047,   15,   15,   87,   45,
           9,   49,  698,  173,  465, 1066,   93,   71, 5373,    3,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   2,  112,  232,  361,  193,  156,   43, 2297,  754,  120,   99,
         709,  196, 4500, 6540,   89,   71,  718, 2776,   42,  376,  371,
          15,   15,   15,   45,    9,   49,  131,  102,  181,   73,  528,
         116,   56, 6558,  273,   72,  468,   75, 1831,   92,    3,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   2,    1,  12

In [ ]:
# This is where the error pops up. I assume it is because of the way we are preparing batches.

# embed_en(encoded)
embed_en(encoded.to_tensor()).shape

TensorShape([5, 68, 512])

### Define the feed forward network

Define a function for the point-wise feed-forward network that you'll use later.

The network consists of two linear layers (`tf.keras.layers.Dense`) with a ReLU activation in-between:

In [ ]:
def point_wise_feed_forward_network(
  d_model, # Input/output dimensionality.
  dff # Inner-layer dimensionality.
  ):

  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
  ])

### Define the encoder layer

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
    super().__init__()


    # Multi-head self-attention.
    self.mha = tf.keras.layers.MultiHeadAttention(
        num_heads=num_attention_heads,
        key_dim=d_model, # Size of each attention head for query Q and key K.
        dropout=dropout_rate,
        )
    # Point-wise feed-forward network.
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    # Layer normalization.
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    # Dropout for the point-wise feed-forward network.
    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, training, mask):

    # A boolean mask.
    if mask is not None:
      mask1 = mask[:, :, None]
      mask2 = mask[:, None, :]
      attention_mask = mask1 & mask2
    else:
      attention_mask = None

    # Multi-head self-attention output (`tf.keras.layers.MultiHeadAttention `).
    attn_output = self.mha(
        query=x,  # Query Q tensor.
        value=x,  # Value V tensor.
        key=x,  # Key K tensor.
        attention_mask=attention_mask, # A boolean mask that prevents attention to certain positions.
        training=training, # A boolean indicating whether the layer should behave in training mode.
        )

    # Multi-head self-attention output after layer normalization and a residual/skip connection.
    out1 = self.layernorm1(x + attn_output)  # Shape `(batch_size, input_seq_len, d_model)`

    # Point-wise feed-forward network output.
    ffn_output = self.ffn(out1)  # Shape `(batch_size, input_seq_len, d_model)`
    ffn_output = self.dropout1(ffn_output, training=training)
    # Point-wise feed-forward network output after layer normalization and a residual skip connection.
    out2 = self.layernorm2(out1 + ffn_output)  # Shape `(batch_size, input_seq_len, d_model)`.

    return out2

In [ ]:
sample_encoder_layer = EncoderLayer(d_model=512, num_attention_heads=8, dff=2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((2, 3, 512)), training=False, mask=None)

# Print the shape.
print(sample_encoder_layer_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

(2, 3, 512)


### Define the encoder

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self,
               *,
               num_layers,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               input_vocab_size, # Input (Portuguese) vocabulary size.
               dropout_rate=0.1
               ):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    # Embeddings + Positional encoding
    self.pos_embedding = PositionalEmbedding(input_vocab_size, d_model)

    # Encoder layers.
    self.enc_layers = [
        EncoderLayer(
          d_model=d_model,
          num_attention_heads=num_attention_heads,
          dff=dff,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    # Dropout.
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  # Masking.
  def compute_mask(self, x, previous_mask=None):
    return self.pos_embedding.compute_mask(x, previous_mask)

  def call(self, x, training):

    seq_len = tf.shape(x)[1]

    # Sum up embeddings and positional encoding.
    mask = self.compute_mask(x)
    x = self.pos_embedding(x)  # Shape `(batch_size, input_seq_len, d_model)`.
    # Add dropout.
    x = self.dropout(x, training=training)

    # N encoder layers.
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    # experimental thing, may not work
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return x  # Shape `(batch_size, input_seq_len, d_model)`.

In [ ]:
# Instantiate the encoder.
sample_encoder = Encoder(
    num_layers=8,
    d_model=512,
    num_attention_heads=8,
    dff=2048,
    input_vocab_size=7010)

# Set the test input.
sample_encoder_output = sample_encoder(encoded.to_tensor(),
                                       training=False)

# Print the shape.
print(encoded.to_tensor().shape)
print(sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

(5, 68)
(5, 1)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(129)),
    sample_encoder
])

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           (None, 1)                 87614464  
                                                                 
Total params: 87,614,464
Trainable params: 87,614,464
Non-trainable params: 0
_________________________________________________________________


In [ ]:
[item for item in dir(model) if not item.startswith('_')]

In [ ]:
model.input_shape

(None, 128)

In [ ]:
tokenized_sentences.to_tensor().shape # iske batches bnane hain...

TensorShape([43012, 129])

In [ ]:
num_layers = 4
d_model = 129
dff = 512
num_attention_heads = 8
dropout_rate = 0.1

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(128)),
    Encoder(num_layers = 4, d_model = 128, dff = 512, num_attention_heads = 8, input_vocab_size=tokenizers.en.get_vocab_size())
])

In [ ]:
input = np.random.rand(1, 128)
# print(input.shape)
target = tf.constant([[1,2,3, 0]])

x = model((input))
x

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.45408416]], dtype=float32)>

In [ ]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         (None, 1)                 3536128   
                                                                 
Total params: 3,536,128
Trainable params: 3,536,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compiling the model

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"]
)

In [ ]:
labels.value_counts()

0    22358
1    20654
Name: Sentiment, dtype: int64

In [ ]:
tokenized_sentences.to_tensor()

<tf.Tensor: shape=(43012, 128), dtype=int64, numpy=
array([[   2,   45,  302, ...,    0,    0,    0],
       [   2,  107,   79, ...,    0,    0,    0],
       [   2, 1366,   31, ...,    0,    0,    0],
       ...,
       [   2,   31,   43, ...,    0,    0,    0],
       [   2,   99,   52, ...,    0,    0,    0],
       [   2, 5681,  551, ...,    0,    0,    0]])>

In [ ]:
list(labels)

In [ ]:
# error!
history = model.fit(
    tokenized_sentences.to_tensor(),
    epochs=200,
    validation_data=labels
)

ValueError: ignored